In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from keras.models import Sequential
import tensorflow as tf
import tensorflow_datasets as tfds

from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
from numpy import *
from PIL import Image

In [9]:
# Define paths and categories
Spath_test = "/content/drive/MyDrive/Crop___Disease/Wheat"
CATEGORIES = [ "Wheat___Brown_Rust", "Wheat___Healthy","Wheat___Yellow_Rust"]
IMG_SIZE = 200
NUM_CLASSES = len(CATEGORIES)

In [ ]:
# Prepare training data
training = []

def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(Spath_test, category)
        print(path)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training.append([new_array, class_num])
            except Exception as e:
                print(f"Error loading image {img}: {e}")

createTrainingData()

/content/drive/MyDrive/Crop___Disease/Wheat/Wheat___Brown_Rust
/content/drive/MyDrive/Crop___Disease/Wheat/Wheat___Healthy
/content/drive/MyDrive/Crop___Disease/Wheat/Wheat___Yellow_Rust


In [ ]:
# Shuffle and split the data
np.random.shuffle(training)
X, y = [], []

for features, label in training:
    X.append(features)
    y.append(label)

In [ ]:
# Convert to NumPy arrays and normalize
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3).astype('float32') / 255.0
y = np.array(y)

In [ ]:
# Convert labels to categorical
from tensorflow.keras.utils import to_categorical
Y = to_categorical(y, NUM_CLASSES)

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [ ]:
# Model parameters
batch_size = 32
nb_epochs = 5

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [ ]:
batch_size = 32
nb_classes = 4
nb_epochs = 5
img_rows, img_columns = 200, 200
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(4,  activation=tf.nn.softmax)
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

Epoch 1/5
74/74 [==============================] - 137s 2s/step - loss: 0.6268 - accuracy: 0.7650 - val_loss: 0.1765 - val_accuracy: 0.9372
Epoch 2/5
74/74 [==============================] - 136s 2s/step - loss: 0.2051 - accuracy: 0.9159 - val_loss: 0.1818 - val_accuracy: 0.9372
Epoch 3/5
74/74 [==============================] - 137s 2s/step - loss: 0.1099 - accuracy: 0.9626 - val_loss: 0.1110 - val_accuracy: 0.9610
Epoch 4/5
74/74 [==============================] - 136s 2s/step - loss: 0.0867 - accuracy: 0.9711 - val_loss: 0.2097 - val_accuracy: 0.9440
Epoch 5/5
74/74 [==============================] - 138s 2s/step - loss: 0.0695 - accuracy: 0.9762 - val_loss: 0.1096 - val_accuracy: 0.9610


In [ ]:
# Save the model manually in .h5 format after training
model.save('Wheat_model.h5')  # Save the model in .h5 format
print("Model saved as 'Wheat_model.h5'.")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



Model saved as 'Wheat_model.h5'.


In [ ]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Test Loss:  0.10962723940610886
Test Accuracy:  0.9609507918357849


In [ ]:
# Plot training & validation accuracy values using Plotly
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['accuracy'], mode='lines+markers', name='Train Accuracy'))
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['val_accuracy'], mode='lines+markers', name='Val Accuracy'))
fig.update_layout(title='Model Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()

In [ ]:
# Plot training & validation loss values using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['loss'], mode='lines+markers', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['val_loss'], mode='lines+markers', name='Val Loss'))
fig.update_layout(title='Model Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

In [ ]:
# Predict an image
image = X_test[0]
image = np.expand_dims(image, axis=0)  # Add batch dimension
predictions = model.predict(image)
predicted_class = np.argmax(predictions[0])
print("Predicted class:", CATEGORIES[predicted_class])


1/1 [==============================] - 0s 125ms/step
Predicted class: Wheat___Brown_Rust


In [7]:
def predict_image(image_path, model):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.astype('float32') / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions[0])

    # Return the class label
    return CATEGORIES[predicted_class]


In [10]:
image_path = "/content/drive/MyDrive/Crop___Disease/Wheat/Wheat___Healthy/Healthy536.jpg"  # Replace with the path to the user-provided image
predicted_disease = predict_image(image_path, model)
print(f"The predicted disease is: {predicted_disease}")


1/1 [==============================] - 0s 156ms/step
The predicted disease is: Wheat___Healthy


In [3]:
from tensorflow.keras.models import load_model
model=load_model("/content/drive/MyDrive/Wheat_model.h5")